<a href="https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/dense_sentiment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dense Sentiment Classifier

In this notebook, we build a dense neural net to classify PE activity.

#### Load dependencies

In [ ]:
import tensorflow
from tensorflow.keras.datasets import imdb # new! 
from tensorflow.keras.preprocessing.sequence import pad_sequences #new!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding # new!
from tensorflow.keras.callbacks import ModelCheckpoint # new! 
import os # new! 
from sklearn.metrics import roc_auc_score, roc_curve # new!
import pandas as pd
import matplotlib.pyplot as plt # new!

#### Set hyperparameters

In [ ]:
# output directory name:
output_dir = 'model_output/dense'
input_dir =  'Z:/Research/dfuller/Walkabilly/studies/smarphone_accel/data/Ethica_Jaeger_Merged/pocket/'
input_file_name = 'pocket_with_couns_and_vec_meg_30Hz.csv'
# training:
epochs = 4
batch_size = 128

# vector-space embedding: 
n_dim = 64
n_unique_words = 5000 # as per Maas et al. (2011); may not be optimal
n_words_to_skip = 50 # ditto
max_review_length = 100
pad_type = trunc_type = 'pre'

# neural network architecture: 
n_dense = 64
dropout = 0.5

#### Load data

##### For this notebook we use the acceleration data gathered from the pocket location.

In [ ]:
df = pd.read_csv(input_dir + input_file_name)


In [9]:
df.head()

,record_time,x_axis,y_axis,z_axis,participant_id,wear_location,V.O2,VO2.kg,V.CO2,V.E,trimmed_activity,height,weight,age,gender,x,y,z,counts_vec_mag
0,2019-01-07T10:30:01Z,0.266111,-0.069464,0.966880,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,Female,13.0,0.0,73.0,74.1485
1,2019-01-07T10:30:01.033300Z,0.266401,-0.069351,0.963832,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,Female,NaN,NaN,NaN,74.1485
2,2019-01-07T10:30:01.066700Z,0.262441,-0.067394,0.943335,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,Female,NaN,NaN,NaN,74.1485
3,2019-01-07T10:30:01.100Z,0.262720,-0.066384,0.940710,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,Female,NaN,NaN,NaN,74.1485
4,2019-01-07T10:30:01.133300Z,0.263644,-0.066399,0.946187,108,pock,NaN,NaN,NaN,NaN,Lying,164.0,68.0,30,Female,NaN,NaN,NaN,74.1485


In [ ]:
x_train[0:6] # 0 reserved for padding; 1 would be starting character; 2 is unknown; 3 is most common word, etc.

In [ ]:
for x in x_train[0:6]:
    print(len(x))

In [ ]:
y_train[0:6]

In [ ]:
len(x_train), len(x_valid)

#### Restoring words from index

In [ ]:
word_index = tensorflow.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["PAD"] = 0
word_index["START"] = 1
word_index["UNK"] = 2

In [ ]:
word_index

In [ ]:
index_word = {v:k for k,v in word_index.items()}

In [ ]:
x_train[0]

In [ ]:
' '.join(index_word[id] for id in x_train[0])

In [ ]:
(all_x_train,_),(all_x_valid,_) = imdb.load_data() 

In [ ]:
' '.join(index_word[id] for id in all_x_train[0])

#### Preprocess data

In [ ]:
x_train = pad_sequences(x_train, maxlen=max_review_length, 
                        padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, 
                        padding=pad_type, truncating=trunc_type, value=0)

In [ ]:
x_train[0:6]

In [ ]:
for x in x_train[0:6]:
    print(len(x))

In [ ]:
' '.join(index_word[id] for id in x_train[0])

In [ ]:
' '.join(index_word[id] for id in x_train[5])

#### Design neural network architecture

In [ ]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(Flatten())
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
# model.add(Dense(n_dense, activation='relu'))
# model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid')) # mathematically equivalent to softmax with two classes

In [ ]:
model.summary() # so many parameters!

In [ ]:
# embedding layer dimensions and parameters: 
n_dim, n_unique_words, n_dim*n_unique_words

In [ ]:
# ...flatten:
max_review_length, n_dim, n_dim*max_review_length

In [ ]:
# ...dense:
n_dense, n_dim*max_review_length*n_dense + n_dense # weights + biases

In [ ]:
# ...and output:
n_dense + 1 

#### Configure model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+
                                  "/weights.{epoch:02d}.hdf5")

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Train!

In [ ]:
model.fit(x_train, y_train, 
         batch_size=batch_size, epochs=epochs, verbose=1, 
         validation_data=(x_valid, y_valid), 
         callbacks=[modelcheckpoint])

# model.fit(x_train, y_train, 
#           batch_size=batch_size, epochs=epochs, verbose=1, 
#           validation_data=(x_valid, y_valid))

#### Evaluate

In [ ]:
model.load_weights(output_dir+"/weights.02.hdf5") # NOT zero-indexed

In [ ]:
y_hat = model.predict_proba(x_valid)

In [ ]:
len(y_hat)

In [ ]:
y_hat[0]

In [ ]:
y_valid[0]

In [ ]:
plt.hist(y_hat)
_ = plt.axvline(x=0.5, color='orange')

In [ ]:
pct_auc = roc_auc_score(y_valid, y_hat)*100.0

In [ ]:
"{:0.2f}".format(pct_auc)

In [ ]:
float_y_hat = []
for y in y_hat:
    float_y_hat.append(y[0])

In [ ]:
ydf = pd.DataFrame(list(zip(float_y_hat, y_valid)), columns=['y_hat', 'y'])

In [ ]:
ydf.head(10)

In [ ]:
' '.join(index_word[id] for id in all_x_valid[0])

In [ ]:
' '.join(index_word[id] for id in all_x_valid[6]) 

In [ ]:
ydf[(ydf.y == 0) & (ydf.y_hat > 0.9)].head(10)

In [ ]:
' '.join(index_word[id] for id in all_x_valid[386]) 

In [ ]:
ydf[(ydf.y == 1) & (ydf.y_hat < 0.1)].head(10)

In [ ]:
' '.join(index_word[id] for id in all_x_valid[224]) 